# Tratamento dos dados
*Objetivos*: 
- Tratar o arquivo NINDINET.DBF para que seja possível a análise dos dados.
- Criar um modelo de previsão de óbitos por Leptospirose

In [1]:
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Funções Auxiliares

In [2]:
# Função para retornar os valores únicos das colunas do df
def valores_unicos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].unique()}')
        print()

# valores_unicos(df[colunas_painel])

# Função para verificar a existência de valores nulos nas colunas do df
def verifica_nulos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].isnull().sum()}')
        print()

# Função para verificar a porcentagem de nulos nas colunas do df
def porcentagem_nulos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].isnull().sum() / len(df) * 100:.2f}%')

# Função para converter a idade
def converter_idade(idade):
    faixas_etarias = {
        (0, 999): 'IGNORADO',
        (1000, 3999): '00 a 01 ano',
        (4000, 4004): '01 a 04 anos',
        (4005, 4009): '05 a 09 anos',
        (4010, 4014): '10 a 14 anos',
        (4015, 4019): '15 a 19 anos',
        (4020, 4029): '20 a 29 anos',
        (4030, 4039): '30 a 39 anos',
        (4040, 4049): '40 a 49 anos',
        (4050, 4059): '50 a 59 anos',
        (4060, 4069): '60 a 69 anos',
        (4070, 4079): '70 a 79 anos',
        (4080, 4999): 'Mais de 80 anos'
    }

    for (inicio, fim), faixa in faixas_etarias.items():
        if inicio <= idade <= fim:
            return faixa
    return 'IGNORADO'

In [3]:
# Todads as colunas do df
colunas_df = ['NU_NOTIFIC', 'TP_NOT', 'ID_AGRAVO', 'CS_SUSPEIT','IN_AIDS', 'CS_MENING',
           'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA',
           'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI', 'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 
           'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_CNS_SUS', 'SG_UF', 'ID_MN_RESI', 
           'ID_RG_RESI', 'ID_DISTRIT', 'ID_BAIRRO', 'NM_BAIRRO', 'NU_CEP', 'NU_DDD_TEL', 
           'NU_TELEFON', 'CS_ZONA', 'ID_PAIS', 'NDUPLIC_N', 'IN_VINCULA', 'DT_INVEST', 
           'ID_OCUPA_N', 'CLASSI_FIN', 'CRITERIO', 'TPAUTOCTO', 'COUFINF', 'COPAISINF',
           'COMUNINF', 'CODISINF', 'CO_BAINFC', 'NOBAIINF', 'DOENCA_TRA', 'EVOLUCAO',
           'DT_OBITO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS', 'DT_TRANSDM', 'DT_TRANSSM',
           'DT_TRANSRM', 'DT_TRANSRS', 'DT_TRANSSE', 'NU_LOTE_V', 'NU_LOTE_H', 'CS_FLXRET', 
           'FLXRECEBI', 'TPUNINOT']

# Colunas que serão analisadas
colunas_painel = ['ID_AGRAVO', 'DT_NOTIFIC', 'DT_SIN_PRI', 'ID_MN_RESI', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'DT_OBITO', 'NU_IDADE_N']

## Convertendo o arquivo .dbf
- Converter o arquivo .xls para .csv

In [4]:
# Converter de .dbf para .csv
# dbf = DBF('dbf/NINDINET.DBF')
# df = pd.DataFrame(iter(dbf))
# df.to_csv('NINDINET.csv', index=False)
# 
# df.head()

- CASO JÁ TENHA CRIADO O ARQUIVO .CSV, PODE EXECUTAR À PARTIR DAQUI
- Descomentar o código abaixo e comentar o código acima

In [5]:
# Carrega o .csv salvo
arquivo_csv = 'NINDINET.csv'

df = pd.read_csv(arquivo_csv, usecols=colunas_painel, sep=',', low_memory=False)

df.head()

,ID_AGRAVO,DT_NOTIFIC,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_MN_RESI,DT_OBITO
0,A90,2007-02-01,2007-01-23,4029,M,6.0,4.0,2.0,150304.0,NaN
1,A90,2007-02-22,2007-02-22,4018,F,9.0,9.0,9.0,150506.0,NaN
2,A90,2008-02-07,2008-02-02,4002,M,6.0,4.0,10.0,150330.0,NaN
3,A90,2008-01-06,2008-01-03,4025,F,5.0,4.0,6.0,150553.0,NaN
4,X29,2008-01-09,2008-01-09,4047,M,6.0,4.0,9.0,150450.0,NaN


## Para a análise, vamos manter apenas os casos confirmados de leptospirose

In [6]:
# Manter somente ID_AGRAVO == A279
df = df[df['ID_AGRAVO'] == 'A279']

# Remover a coluna ID_AGRAVO
df = df.drop(columns=['ID_AGRAVO'])

In [7]:
total_de_casos = len(df)

print(f'Temos um total de {total_de_casos} casos de leptospirose.')

Temos um total de 13353 casos de leptospirose.


## Tratamento de valores nulos

In [8]:
porcentagem_nulos(df)

DT_NOTIFIC: 0.00%
DT_SIN_PRI: 0.00%
NU_IDADE_N: 0.00%
CS_SEXO: 0.00%
CS_GESTANT: 0.00%
CS_RACA: 4.43%
CS_ESCOL_N: 16.45%
ID_MN_RESI: 0.02%
DT_OBITO: 95.63%


### Utilizar o KNNImputer para preencher os valores nulos

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13353 entries, 2261 to 1661182
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  13353 non-null  object 
 1   DT_SIN_PRI  13353 non-null  object 
 2   NU_IDADE_N  13353 non-null  int64  
 3   CS_SEXO     13353 non-null  object 
 4   CS_GESTANT  13353 non-null  float64
 5   CS_RACA     12762 non-null  float64
 6   CS_ESCOL_N  11156 non-null  float64
 7   ID_MN_RESI  13350 non-null  float64
 8   DT_OBITO    583 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 1.0+ MB


In [10]:
imputer = KNNImputer(n_neighbors=10)

# Utilizar o KNNImputer para preencher os valores nulos da coluna CS_RACA
df['CS_RACA'] = imputer.fit_transform(df[['CS_RACA']])

# Utilizar o KNNImputer para preencher os valores nulos da coluna CS_ESCOL_N
df['CS_ESCOL_N'] = imputer.fit_transform(df[['CS_ESCOL_N']])

### Remover os registros das colunas com baixa quantidade de valores nulos

In [11]:
# Removendo as linhas nulas na coluna ID_MN_RESI
df = df.dropna(subset=['ID_MN_RESI'])

In [12]:
verifica_nulos(df)

DT_NOTIFIC: 0

DT_SIN_PRI: 0

NU_IDADE_N: 0

CS_SEXO: 0

CS_GESTANT: 0

CS_RACA: 0

CS_ESCOL_N: 0

ID_MN_RESI: 0

DT_OBITO: 12768



In [13]:
# Transformar a coluna DT_OBITO em booleana
df['DT_OBITO'] = df['DT_OBITO'].notnull().astype(int)

In [14]:
# Transformmar as colunas DT_NOTIFIC e DT_SIN_PRI em datetime
df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'], format='%Y-%m-%d')
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'], format='%Y-%m-%d')

In [15]:
# Criar uma coluna chamada NU_DIAS entre a data de notificação e a data dos primeiros sintomas
df['NU_DIAS'] = (df['DT_NOTIFIC'] - df['DT_SIN_PRI']).dt.days
df.sample(5)

,DT_NOTIFIC,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_MN_RESI,DT_OBITO,NU_DIAS
172994,2016-01-03,2015-12-28,4011,M,6.0,4.0,2.0,150140.0,1,6
709417,2012-05-22,2012-05-15,4017,F,5.0,4.0,4.0,150240.0,0,7
146276,2012-02-03,2012-02-01,4059,M,6.0,4.0,2.0,150240.0,0,2
476611,2015-03-13,2015-03-07,4018,F,5.0,1.0,9.0,150340.0,0,6
1452844,2007-11-18,2007-10-19,4046,M,6.0,4.0,2.0,150140.0,0,30


In [16]:
# Remover as colunas DT_NOTIFIC e DT_SIN_PRI
df = df.drop(columns=['DT_NOTIFIC', 'DT_SIN_PRI'])

In [17]:
# Convertar a coluna NU_IDADE_N
df['FAIXA_ETARIA'] = df['NU_IDADE_N'].apply(converter_idade)

# Excluir a coluna NU_IDADE_N
df = df.drop(columns=['NU_IDADE_N'])

In [18]:
# Converter as colunas CS_SEXO, CS_GESTANT, CS_RACA, CS_ESCOL_N, ID_MN_RESI, FAIXA_ETARIA em variáveis dummies
df2 = pd.get_dummies(df, columns=['CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_MN_RESI', 'FAIXA_ETARIA'], drop_first=True)

# Aplicar Regressão Logística 

In [19]:
# Dividir o df em treino e teste, levando em consideração a coluna DT_OBITO e o desbalanceamento dos dados
X = df2.drop(columns=['DT_OBITO'])
y = df2['DT_OBITO']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [20]:
# Regressão Logística com ajuste de pesos para lidar com classes desbalanceadas
logreg_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
logreg_balanced.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [21]:
# Fazendo previsões e avaliando o modelo
y_pred_balanced = logreg_balanced.predict(X_test)
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
conf_matrix_balanced = confusion_matrix(y_test, y_pred_balanced)
class_report_balanced = classification_report(y_test, y_pred_balanced)

print(f'Acurácia {accuracy_balanced}')
print(f'Matriz de Confusão:\n {conf_matrix_balanced}')
print(class_report_balanced)


Acurácia 0.6868913857677903
Matriz de Confusão:
 [[1740  814]
 [  22   94]]
              precision    recall  f1-score   support

           0       0.99      0.68      0.81      2554
           1       0.10      0.81      0.18       116

    accuracy                           0.69      2670
   macro avg       0.55      0.75      0.49      2670
weighted avg       0.95      0.69      0.78      2670



# Aplicar Random Forest

In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print(f'Acurácia {accuracy_rf}')
print(f'Matriz de Confusão:\n {conf_matrix_rf}')
print(class_report_rf)



Acurácia 0.9385767790262173
Matriz de Confusão:
 [[2498   56]
 [ 108    8]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2554
           1       0.12      0.07      0.09       116

    accuracy                           0.94      2670
   macro avg       0.54      0.52      0.53      2670
weighted avg       0.92      0.94      0.93      2670



# Aplicar SMOTE para balancear as classes

In [23]:
# Aplicar o SMOTE no conjunto de treino
smote = SMOTE(k_neighbors=20, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [24]:
y_test.value_counts()

DT_OBITO
0    2554
1     116
Name: count, dtype: int64

In [25]:
# Verificar a quantidade de amostras de cada classe
y_train_smote.value_counts()


DT_OBITO
0    10214
1    10214
Name: count, dtype: int64

In [26]:
# Treinar o modelo de Regressão Logísitica após aplicar SMOTE
logreg_smote = LogisticRegression(max_iter=1000, solver='saga', penalty='elasticnet', l1_ratio=0.4)
logreg_smote.fit(X_train_smote, y_train_smote)

# Fazer previsões no conjunto de teste
y_pred_smote = logreg_smote.predict(X_test)

# Avaliar o desempenho
accuracy_smote = accuracy_score(y_test, y_pred_smote)
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
class_report_smote = classification_report(y_test, y_pred_smote)

# Exibir os resultados
print(f"Acurácia: {accuracy_smote}")
print(f"Matriz de Confusão:\n{conf_matrix_smote}")
print(f"Relatório de Classificação:\n{class_report_smote}")

Acurácia: 0.8213483146067416
Matriz de Confusão:
[[2149  405]
 [  72   44]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.84      0.90      2554
           1       0.10      0.38      0.16       116

    accuracy                           0.82      2670
   macro avg       0.53      0.61      0.53      2670
weighted avg       0.93      0.82      0.87      2670



C:\Users\ricme\anaconda3\envs\ebac\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [27]:
# Treinar o modelo Random Forest após o SMOTE
rf_smote = RandomForestClassifier(n_estimators=500, random_state=42, min_samples_leaf=15)
rf_smote.fit(X_train_smote, y_train_smote)

# Fazer previsões no conjunto de teste
y_pred_rf_smote = rf_smote.predict(X_test)

# Avaliar o desempenho
accuracy_rf_smote = accuracy_score(y_test, y_pred_rf_smote)
conf_matrix_rf_smote = confusion_matrix(y_test, y_pred_rf_smote)
class_report_rf_smote = classification_report(y_test, y_pred_rf_smote)



# Exibir os resultados
print(f"Acurácia: {accuracy_rf_smote}")
print(f"Matriz de Confusão:\n{conf_matrix_rf_smote}")
print(f"Relatório de Classificação:\n{class_report_rf_smote}")

Acurácia: 0.8951310861423221
Matriz de Confusão:
[[2347  207]
 [  73   43]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.92      0.94      2554
           1       0.17      0.37      0.23       116

    accuracy                           0.90      2670
   macro avg       0.57      0.64      0.59      2670
weighted avg       0.94      0.90      0.91      2670



# Aplicar Under Sampling



In [28]:
# Instanciar o RandomUnderSampler
undersampler = RandomUnderSampler(random_state=42)

# Aplicar o undersampling apenas no conjunto de treino
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [29]:
# Treinar o modelo de Regressão Logísitica após aplicar SMOTE
logreg_smote = LogisticRegression(max_iter=1000, solver='saga', penalty='elasticnet', l1_ratio=0.5)
logreg_smote.fit(X_train_under, y_train_under)

# Fazer previsões no conjunto de teste
y_pred_under = logreg_smote.predict(X_test)

# Avaliar o desempenho
accuracy_smote = accuracy_score(y_test, y_pred_under)
conf_matrix_smote = confusion_matrix(y_test, y_pred_under)
class_report_smote = classification_report(y_test, y_pred_under)

# Exibir os resultados
print(f"Acurácia: {accuracy_smote}")
print(f"Matriz de Confusão:\n{conf_matrix_smote}")
print(f"Relatório de Classificação:\n{class_report_smote}")

Acurácia: 0.5883895131086142
Matriz de Confusão:
[[1492 1062]
 [  37   79]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.98      0.58      0.73      2554
           1       0.07      0.68      0.13       116

    accuracy                           0.59      2670
   macro avg       0.52      0.63      0.43      2670
weighted avg       0.94      0.59      0.70      2670



C:\Users\ricme\anaconda3\envs\ebac\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [30]:
# Instanciar o modelo Random Forest
rf = RandomForestClassifier(n_estimators=500, min_samples_leaf=15, random_state=42)

# Treinar o modelo com os dados subamostrados
rf.fit(X_train_under, y_train_under)

# Fazer previsões no conjunto de teste
y_pred = rf.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.6569288389513108
Matriz de Confusão:
[[1665  889]
 [  27   89]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.98      0.65      0.78      2554
           1       0.09      0.77      0.16       116

    accuracy                           0.66      2670
   macro avg       0.54      0.71      0.47      2670
weighted avg       0.95      0.66      0.76      2670

